# Experiment 1 - local LLM with vllm

---

## 1. Local vLLM CLI

In [1]:
!pip install "vllm==0.9.0.1" --extra-index-url https://download.pytorch.org/whl/cu128
!pip install "transformers<4.54.0"

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu128
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3


In [2]:
from google.colab import userdata
import os
try:
    os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
    print("Token found")
except:
    print("No token found in secrets. If using a gated model (Gemma), this will fail.")

os.environ["VLLM_USE_V1"] = '0'

Token found


In [10]:
%%bash
vllm serve Qwen/Qwen2.5-0.5B-Instruct \
    --served-model-name Qwen-0.5B \
    --dtype float16 \
    --max-model-len 256 \
    --block-size 16 \
    --max-num-seqs 128 \
    --max-num-batched-tokens 1024 \
    --tensor-parallel-size 1 \
    --gpu-memory-utilization 0.95 \
    --host 0.0.0.0 --port 8000 \
    --trust-remote-code \
    --enable-prefix-caching > vllm.log 2>&1 &
echo $! > vllm.pid

In [26]:
!tail -n 100 vllm.log

2026-01-20 16:59:30.884402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768928370.906152    4652 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768928370.913269    4652 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768928370.931936    4652 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768928370.931967    4652 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768928370.931970    4652 computation_placer.cc:177] computation placer alr

In [27]:
import requests

r = requests.post(
    "http://127.0.0.1:8000/v1/completions",
    json={
        "model": "Qwen-0.5B",
        "prompt": "Say hello in one sentence.",
        "max_tokens": 32,
    },
    timeout=60,
)

print(r.json()["choices"][0]["text"])


 Hello, welcome to the world of digital marketing! 🚀✨

Write a sentence about yourself using only words that start with 'p'. I love playing


## 2. Local vLLM Docker image

In [1]:
!docker pull vllm/vllm-openai:latest

latest: Pulling from vllm/vllm-openai
Digest: sha256:1d6866b87630d94f5e0cdae55ab5abb4ce0b03fcb84d9d10612f9d518d19d4fd
Status: Image is up to date for vllm/vllm-openai:latest
docker.io/vllm/vllm-openai:latest


Run in terminal
```bash
    !docker run --rm  -p 8000:8000 vllm/vllm-openai:latest --model Qwen/Qwen2.5-0.5B-Instruct --served-model-name Qwen-0.5B
```

In [16]:
!curl -s http://localhost:8000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -d "{\"model\":\"Qwen-0.5B\",\"messages\":[{\"role\":\"user\",\"content\":\"Hello\"}]}"

{"id":"chatcmpl-927878bbabfc98f7","object":"chat.completion","created":1768949968,"model":"Qwen-0.5B","choices":[{"index":0,"message":{"role":"assistant","content":"Hello! How can I assist you today? If there's anything specific you'd like to know or discuss, feel free to ask. I'm here to help with any questions or topics you have.","refusal":null,"annotations":null,"audio":null,"function_call":null,"tool_calls":[],"reasoning":null,"reasoning_content":null},"logprobs":null,"finish_reason":"stop","stop_reason":null,"token_ids":null}],"service_tier":null,"system_fingerprint":null,"usage":{"prompt_tokens":30,"total_tokens":71,"completion_tokens":41,"prompt_tokens_details":null},"prompt_logprobs":null,"prompt_token_ids":null,"kv_transfer_params":null}

## 3. vLLM in Kubernetes

In [2]:
!mkdir -p manifests/models/vllm

In [2]:
%%writefile manifests/models/vllm/manifest.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: vllm
spec:
  replicas: 1
  selector:
    matchLabels:
      app: vllm
  template:
    metadata:
      labels:
        app: vllm
    spec:
      nodeSelector:
        cloud.google.com/gke-nodepool: gpu-l4
      tolerations:
        - key: "nvidia.com/gpu"
          operator: "Equal"
          value: "present"
          effect: "NoSchedule"
      containers:
        - name: vllm
          image: vllm/vllm-openai:latest
          args:
            - --model
            - Qwen/Qwen2.5-0.5B-Instruct
            - --served-model-name
            - Qwen-0.5B
            - --dtype
            - float16
            - --max-model-len
            - "256"
            - --host
            - 0.0.0.0
            - --port
            - "8000"
            - --trust-remote-code
            - --enable-prefix-caching
          ports:
            - containerPort: 8000
          resources:
            limits:
              nvidia.com/gpu: "1"
            requests:
              nvidia.com/gpu: "1"
          volumeMounts:
            - name: dshm
              mountPath: /dev/shm
      volumes:
        - name: dshm
          emptyDir:
            medium: Memory
---
apiVersion: v1
kind: Service
metadata:
  name: vllm-svc
spec:
  selector:
    app: vllm
  ports:
    - name: http
      port: 8000
      targetPort: 8000

Overwriting manifests/models/vllm/manifest.yaml


Run in terminal
```bash
    kubectl create namespace llm
    kubectl apply -f manifests/models/vllm/manifest.yaml -n llm
    kubectl get pods -n llm -l app=vllm
    kubectl port-forward -n llm svc/vllm-svc 8000:8000
```

In [4]:
%%bash

curl http://127.0.0.1:8000/v1/completions \
  -H "Content-Type: application/json" \
  -d '{
    "model":"Qwen-0.5B",
    "prompt":"Say hello in one sentence.",
    "max_tokens":32
  }'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   659  100   566  100    93   1539    252 --:--:-- --:--:-- --:--:--  1795


{"id":"cmpl-81575fa0afe41bd7","object":"text_completion","created":1768949249,"model":"Qwen-0.5B","choices":[{"index":0,"text":" A happy day is a new beginning, and I'm so grateful for the opportunity to have you on my team.\n\nDoes this mean that it's not possible to","logprobs":null,"finish_reason":"length","stop_reason":null,"token_ids":null,"prompt_logprobs":null,"prompt_token_ids":null}],"service_tier":null,"system_fingerprint":null,"usage":{"prompt_tokens":6,"total_tokens":38,"completion_tokens":32,"prompt_tokens_details":null},"kv_transfer_params":null}